## Data

In [1]:
import torch
import torch.nn as nn
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "i like cats and dogs",
    "books are expensive"    
]
data_size = len(corpus)

# 0: noun/pronoun - 1: verb - others - 2
labels = [[0, 1, 0, 2, 0],
          [0, 1, 2]]

# Define the max vocabulary size and sequence length
vocab_size = 10
sequence_length = 5

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'dogs': 6,
 'books': 4,
 'like': 9,
 'i': 8,
 'cats': 5,
 'are': 3,
 'expensive': 7,
 'and': 2,
 '<pad>': 1,
 '<unk>': 0}

In [3]:
# Tokenize and numericalize your samples
def vectorize(text, vocab, sequence_length, sequence_label):
    tokens = tokenizer(text)
    
    token_ids = [vocab[token] for token in tokens]    
    token_ids = token_ids + [vocab["<pad>"]] * (sequence_length - len(tokens))
    sequence_label = sequence_label + [3] * (sequence_length - len(tokens))
    
    return torch.tensor(token_ids, dtype=torch.long), torch.tensor(sequence_label, dtype=torch.long)

# Vectorize the samples
sentence_vecs = []
label_vecs = []
for sentence, labels in zip(corpus, labels):
    sentence_vec, labels_vec = vectorize(sentence, vocab, sequence_length, labels)
    sentence_vecs.append(sentence_vec)
    label_vecs.append(labels_vec)

In [4]:
for v in sentence_vecs:
    print(v)

tensor([8, 9, 5, 2, 6])
tensor([4, 3, 7, 1, 1])


In [5]:
for v in label_vecs:
    print(v)

tensor([0, 1, 0, 2, 0])
tensor([0, 1, 2, 3, 3])


## Model

In [6]:
class POS_Model(nn.Module):
    def __init__(self, vocab_size, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 4)
        self.fc = nn.Linear(4, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.fc(x)
        return x.permute(0, 2, 1)

model = POS_Model(vocab_size, 4)

# test
data = torch.tensor([[3, 2, 3, 2, 1]]).long()
output = model(data)
print(output.shape)

torch.Size([1, 4, 5])


## Train with full data

In [7]:
criterion = nn.CrossEntropyLoss(ignore_index=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [8]:
input_data = torch.tensor( [[8, 9, 5, 2, 6],
                            [4, 3, 7, 1, 1]], dtype=torch.long)
label_data = torch.tensor([[0, 1, 0, 2, 0],
                           [0, 1, 2, 3, 3]], dtype=torch.long)

for _ in range(50):
    optimizer.zero_grad()
    outputs = model(input_data)
    loss = criterion(outputs, label_data)
    print(loss.item())
    loss.backward()
    optimizer.step()

1.9935970306396484
1.947340726852417
1.9019087553024292
1.8572534322738647
1.8133065700531006
1.7700202465057373
1.7273814678192139
1.6854008436203003
1.6440976858139038
1.603490948677063
1.563600778579712
1.5244474411010742
1.4860503673553467
1.4484264850616455
1.4115923643112183
1.3755643367767334
1.3403557538986206
1.305978536605835
1.2724426984786987
1.2397559881210327
1.2079236507415771
1.1769479513168335
1.1468288898468018
1.11756432056427
1.0891485214233398
1.0615731477737427
1.0348272323608398
1.0088964700698853
0.9837644100189209
0.9594123959541321
0.9358193874359131
0.9129630923271179
0.8908193707466125
0.869363009929657
0.84856778383255
0.8284067511558533
0.8088524341583252
0.7898773550987244
0.7714535593986511
0.7535534501075745
0.7361491322517395
0.7192135453224182
0.7027193903923035
0.6866397261619568
0.6709482669830322
0.6556193232536316
0.6406277418136597
0.625948965549469
0.6115602254867554
0.5974390506744385


In [9]:
outputs = model(input_data)
o_softmax = torch.softmax(outputs, axis=1)

print(o_softmax[0, :, 0].detach().numpy())
print(o_softmax[0, :, 1].detach().numpy())
print(o_softmax[0, :, 2].detach().numpy())
print(o_softmax[0, :, 3].detach().numpy())
print(o_softmax[0, :, 4].detach().numpy())

print()
print(o_softmax[1, :, 0].detach().numpy())
print(o_softmax[1, :, 1].detach().numpy())
print(o_softmax[1, :, 2].detach().numpy())
print(o_softmax[1, :, 3].detach().numpy())
print(o_softmax[1, :, 4].detach().numpy())

[0.92500716 0.01751258 0.01810668 0.03937361]
[0.2644777  0.26781186 0.37938598 0.08832446]
[0.6092647  0.05398951 0.29893586 0.03780996]
[0.13897136 0.33722597 0.45865643 0.06514626]
[0.85315484 0.04864125 0.03080858 0.06739539]

[0.5946117  0.12447325 0.1937442  0.08717094]
[0.04731185 0.6562438  0.21063732 0.08580703]
[0.21985601 0.29077664 0.40728015 0.08208719]
[0.09596418 0.28715825 0.5516926  0.06518501]
[0.09596418 0.28715825 0.5516926  0.06518501]


In [10]:
# labels = [[0, 1, 0, 2, 0],
#           [0, 1, 2, *, *]]

o_softmax.argmax(axis=1)

tensor([[0, 2, 0, 2, 0],
        [0, 1, 2, 2, 2]])